In [428]:
#import thư viện
import pandas as pd
import numpy as np
import time
import datetime

<h1> Lưu ý </h1>
Trong bảng demand và schedule receipt mỗi mỗi một ngày chỉ được phát sinh 1 bản ghi của nguyên liệu hoặc thành phẩm nào đó

In [429]:
#import data vào các dataframe
df_demand = pd.read_excel('C:\Study\MRP python\DATA_ROUND2_SCHACKFEST2022.xlsx', sheet_name='1_DEMAND')
df_BOM = pd.read_excel('C:\Study\MRP python\DATA_ROUND2_SCHACKFEST2022.xlsx', sheet_name='2_BOM')
df_warehouse = pd.read_excel('C:\Study\MRP python\DATA_ROUND2_SCHACKFEST2022.xlsx', sheet_name='3_SUPPLY-SOH')
df_ongoing = pd.read_excel('C:\Study\MRP python\DATA_ROUND2_SCHACKFEST2022.xlsx', sheet_name='4_SUPPLY-ONGOING')
df_materialMaster = pd.read_excel('C:\Study\MRP python\DATA_ROUND2_SCHACKFEST2022.xlsx', sheet_name='5_ITEM MASTER - RAW')
df_productMaster = pd.read_excel('C:\Study\MRP python\DATA_ROUND2_SCHACKFEST2022.xlsx', sheet_name='6_ITEM MASTER - FG')


In [430]:
df_productMaster.rename(columns={'LEADTIME SẢN XUẤT \n(NGÀY)':'LEADTIME SẢN XUẤT(NGÀY)', 
                            'SHELF LIFE KỂ TỪ NGÀY SẢN XUẤT\n(NGÀY)':'SHELF LIFE KỂ TỪ NGÀY SẢN XUẤT(NGÀY)'}, inplace=True)
df_productMaster.head()

,SKU,LOẠI,LEADTIME SẢN XUẤT(NGÀY),SHELF LIFE KỂ TỪ NGÀY SẢN XUẤT(NGÀY)
0,THỨC_ĂN_CHO_HEO,Thành phẩm,0,60
1,THỨC_ĂN_CHO_CÁ,Thành phẩm,0,60


In [431]:
df_materialMaster.set_index(keys=['ITEM', 'NHÀ CUNG CẤP'], inplace=True)

In [432]:
df_warehouse.rename(columns={'TỒN KHO CUỐI NGÀY 31/12/2022':'LƯỢNG TỒN KHO'}, inplace=True)


In [433]:
df_ongoing.set_index(keys=['NGUYÊN LIỆU', 'NGÀY VỀ'], inplace=True)

In [434]:
#unpivot demand
df_demand_unpivot = pd.melt(df_demand, id_vars='ITEM')
df_demand_unpivot.rename(columns={'variable':'demand_date'}, inplace=True)
df_demand_unpivot.head()

,ITEM,demand_date,value
0,THỨC_ĂN_CHO_HEO,JAN_2023,18
1,THỨC_ĂN_CHO_CÁ,JAN_2023,16
2,THỨC_ĂN_CHO_HEO,FEB_2023,22
3,THỨC_ĂN_CHO_CÁ,FEB_2023,21
4,THỨC_ĂN_CHO_HEO,MAR_2023,30


In [435]:
#Define function để chuyển đổi demand date thành dạng datetime
def convert_date(x):
    
    return datetime.datetime(year=int(x[x.find('_') + 1:len(x)]), month=int(time.strptime(x[0:x.find('_')],'%b').tm_mon), day=1)

df_demand_unpivot['demand_date'] = df_demand_unpivot['demand_date'].apply(convert_date)

In [436]:
#Đổi tên lại cột trong bảng BOM
df_BOM.rename(columns={'SỐ LƯỢNG NGUYÊN LIỆU \n(TRÊN 1 TẤN THÀNH PHẨM)':'SỐ LƯỢNG NGUYÊN LIỆU'}, inplace=True)
df_BOM.head()

,THÀNH PHẨM,NGUYÊN LIỆU,SỐ LƯỢNG NGUYÊN LIỆU,ĐƠN VỊ CỦA NGUYÊN LIỆU
0,THỨC_ĂN_CHO_HEO,NGÔ,400,kg
1,THỨC_ĂN_CHO_HEO,KHOAI,200,kg
2,THỨC_ĂN_CHO_HEO,SẮN,200,kg
3,THỨC_ĂN_CHO_HEO,XƯƠNG_ĐỘNG_VẬT,200,kg
4,THỨC_ĂN_CHO_HEO,MEN_VI_SINH,10,lít


In [437]:
#Define ra các biến input đầu vào để chạy báo cáo
dummy_date_input = '2022-01-01'
dummy_date = datetime.datetime.strptime(dummy_date_input, '%Y-%m-%d')

start_date_input = input('Nhập ngày bắt đầu yyyy-mm-dd')
start_date = datetime.datetime.strptime(start_date_input, '%Y-%m-%d')

end_date_input = input('Nhập ngày kết thúc yyyy-mm-dd')
end_date = datetime.datetime.strptime(end_date_input, '%Y-%m-%d')

plan_product = input('Nhập thành phẩm cần phải lên kế hoạch')

List_of_material = list(df_BOM.loc[df_BOM['THÀNH PHẨM'] == plan_product]['NGUYÊN LIỆU'])

plan_material = input('Nhập tên nguyên liệu muốn thực hiện MRP ' + str(List_of_material))



In [438]:
    #Tạo ra một dataframe để tính MRP với ngày bắt đầu là ngày được khai báo trong startdate

MRP_dict = {
'Attribute':['Gross requirements', 'Scheduled Receipt', 'Projected on hand', 'Net requirement', 'Planned Receipt', 'Planned Release'],
'MRP_date':[dummy_date, dummy_date, dummy_date, dummy_date, dummy_date, dummy_date],
'MRP_value':[None, None, None, None, None, None]
}

df_MRP = pd.DataFrame(MRP_dict)

#Tạo một dataframe rỗng chứa daily demand do người dùng nhập
df_daily_product_demand = pd.DataFrame(columns = ['Thành phẩm', 'Demand date', 'Demand value'])

#Để người dùng input daily demand vào trong dataframe
a=0

while a != 'Q':
        a = input('Nhấn Q nếu không muốn nhập nhu cầu theo ngày nữa \n Nhấn phím bất kì để tiếp tục').upper()
        if a == 'Q':
            break
        
        input_demand_date = input('Nhập ngày phát sinh nhu cầu theo định dạng (yyyy-MM-dd)')
        input_demand_date_2 = datetime.datetime.strptime(input_demand_date, '%Y-%m-%d')

        

        input_demand_value = int(input('Số lượng phát sinh nhu cầu ? (Nhập số nguyên)'))

        df_daily_product_demand = df_daily_product_demand.append(
            {'Thành phẩm':plan_product, 
            'Demand date':input_demand_date_2, 
            'Demand value':input_demand_value}, 
            ignore_index=True
            )



#Join vào để tạo ra bảng material demand daily
df_material_demand_daily = df_BOM.merge(df_daily_product_demand, how='right', right_on='Thành phẩm', left_on='THÀNH PHẨM')
del(df_material_demand_daily['Thành phẩm'])
df_material_demand_daily['Daily material demand'] = df_material_demand_daily['Demand value'] * df_material_demand_daily['SỐ LƯỢNG NGUYÊN LIỆU']
df_material_demand_daily.set_index(keys=['THÀNH PHẨM', 'NGUYÊN LIỆU', 'Demand date'], inplace=True)


#Input các dữ liệu về demand vào trong MRP
try:
        df_input_demand = df_material_demand_daily.loc[plan_product, plan_material].reset_index()[['Demand date', 'Daily material demand']]
        for lab, row in df_input_demand.iterrows():
            df_MRP = df_MRP.append({'Attribute':'Gross requirements', 'MRP_date':row['Demand date'], 'MRP_value':row['Daily material demand']}, ignore_index=True)
except KeyError: print('Không tìm thấy nhu cầu')

#Input các dữ liệu về hàng sắp về vào MRP
try:
        df_input_ongoing = df_ongoing.loc[plan_material].reset_index()[['NGÀY VỀ', 'SỐ LƯỢNG']]
        for lab, row in df_input_ongoing.iterrows():
            df_MRP = df_MRP.append({'Attribute':'Scheduled Receipt', 'MRP_date':row['NGÀY VỀ'], 'MRP_value':row['SỐ LƯỢNG']}, ignore_index=True) 
except KeyError: 
        print('Không có hàng nào đang về')
        pass

#Drop dummy date
df_MRP.drop(df_MRP[df_MRP['MRP_date'] == '2022-01-01'].index, inplace=True)


df_MRP








C:\Users\admin\AppData\Local\Temp\ipykernel_14372\4143257106.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_daily_product_demand = df_daily_product_demand.append(
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\4143257106.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_daily_product_demand = df_daily_product_demand.append(
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\4143257106.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_daily_product_demand = df_daily_product_demand.append(
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\4143257106.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_daily_produ

,Attribute,MRP_date,MRP_value
6,Gross requirements,2023-01-01,6000
7,Gross requirements,2023-01-03,6800
8,Gross requirements,2023-01-05,8000
9,Gross requirements,2023-01-07,6800
10,Gross requirements,2023-01-10,6000
11,Scheduled Receipt,2023-01-05,1000


In [439]:
#Bỏ đi những hàng bị hết hạn sử dụng
df_warehouse.drop(df_warehouse[df_warehouse['HẠN SỬ DỤNG'] == start_date].index, inplace=True)

#Tính toán onhand, net requirement, planned receipt, planned release
#Tính toán inventory on hand và thêm vào bảng MRP
while start_date != end_date + datetime.timedelta(days=1):
    try:
        inventory = df_warehouse[df_warehouse['ITEM'] == plan_material]['LƯỢNG TỒN KHO'].sum()
    except KeyError:
        print('Không có hàng {} nào tồn trong kho'.format(plan_material))
        inventory = 0
    df_MRP = df_MRP.append({'Attribute':'Inventory On-hand', 'MRP_date':start_date, 'MRP_value':inventory}, ignore_index=True)

    #Tìm xem có giá trị demand hôm nay không, nếu không thì demand tức thời = 0
    try:
        demand = df_MRP[df_MRP['Attribute'] == 'Gross requirements'][df_MRP['MRP_date'] == start_date]['MRP_value'].item()
    except:
        demand = 0
        df_MRP = df_MRP.append({'Attribute':'Gross requirements', 'MRP_date':start_date, 'MRP_value':0}, ignore_index=True)


    #Thêm dòng net requirement vào trong bảng
    net_requirements = demand - inventory
    df_MRP = df_MRP.append({'Attribute':'Net Requirements', 'MRP_date':start_date, 'MRP_value':net_requirements if net_requirements >= 0 else 0}, ignore_index=True)


    #Thêm dòng planned receipt với điều kiện đặt hàng MOQ
    material_MOQ = df_materialMaster.loc[plan_material]['MOQ'].sum()
    if net_requirements > 0 and net_requirements <= material_MOQ:
        df_MRP = df_MRP.append({'Attribute':'Planned Receipt', 'MRP_date':start_date, 'MRP_value':material_MOQ}, ignore_index=True) 
        material_schedule_release = material_MOQ
    elif net_requirements > 0 and net_requirements > material_MOQ:
        df_MRP = df_MRP.append({'Attribute':'Planned Receipt', 'MRP_date':start_date, 'MRP_value':net_requirements}, ignore_index=True)
        material_schedule_release = net_requirements
    else:
        df_MRP = df_MRP.append({'Attribute':'Planned Receipt', 'MRP_date':start_date, 'MRP_value':0}, ignore_index=True)
        material_schedule_release = 0

    #Thêm dòng schedule release
    material_leadtime = df_materialMaster.loc[plan_material]['LEADTIME ĐẶT HÀNG (NGÀY)'].item()
    df_MRP = df_MRP.append({'Attribute':'Schedule Releases', 'MRP_date':start_date - datetime.timedelta(days=material_leadtime), 'MRP_value':material_schedule_release}, ignore_index=True)


    #Cập nhập thêm mới hàng tồn kho vào trong bảng warehouse

    #Lọc ra các planned receipt và schedule receipt mà trong ngày đang được xét, tính tổng lượng nhập hàng
    warehouse_import_quantity  = df_MRP[df_MRP['MRP_date'] == start_date][df_MRP['Attribute'].isin(['Scheduled Receipt', 'Planned Receipt'])]['MRP_value'].sum()

    #Lấy ra shelf life và đơn vị của hàng hóa được nhập vào
    material_shelf_life = df_materialMaster.loc[plan_material]['SHELF LIFE ĐƯỢC ƯỚC TÍNH KỂ TỪ NGÀY NHẬP KHO (NGÀY)'].item()
    material_unit = df_materialMaster.loc[plan_material]['Đơn vị'].item()
    #Dựa vào shelf life tìm ra hạn sử dụng hàng hóa
    material_expired_date = start_date + datetime.timedelta(days=material_shelf_life)

    #Thêm dòng dữ liệu trong bảng tồn kho nếu lượng nhập vào khác 0
    if warehouse_import_quantity != 0:
        df_warehouse = df_warehouse.append({'ITEM':plan_material, 'LƯỢNG TỒN KHO':warehouse_import_quantity, 'ĐƠN VỊ':material_unit, 'HẠN SỬ DỤNG':material_expired_date, 'NGÀY NHẬP KHO':start_date}, ignore_index=True)






    #Xuất kho những mặt hàng theo quy tắc FIFO nếu trung bình số ngày tồn kho > 30
    #Tính ra trung bình ngày tồn các mặt hàng có trong kho
    df_warehouse['NGÀY ĐÃ TỒN KHO'] = start_date - df_warehouse['NGÀY NHẬP KHO']

    cumulative_demand = 0
    #Xuất FIFO cho đến khi trung bình ngày tồn kho <= 30
    if df_warehouse['NGÀY ĐÃ TỒN KHO'].dt.days.mean() > 30:
        #Lọc ra plan material, sắp xếp thấp dần theo ngày đã tồn kho và tiến hành trừ tồn kho
        for lab, row in df_warehouse[df_warehouse['ITEM'] == plan_material].sort_values('NGÀY ĐÃ TỒN KHO', ascending=False).iterrows():

            #Check sau khi trừ tồn xong thì trung bình tồn kho đã dưới 30 ngày chưa, nếu rồi thì thoát vòng lặp
            if df_warehouse['NGÀY ĐÃ TỒN KHO'].dt.days.mean() <= 30:
                break

            #Nếu mà demand tích lũy <= lượng tồn kho thì tiến hành xóa dòng đó đi
            if row['LƯỢNG TỒN KHO'] + cumulative_demand <= demand:
                print(lab)
                df_warehouse.drop(lab, inplace=True)
                cumulative_demand = cumulative_demand + row['LƯỢNG TỒN KHO']
            #Nếu không thì tiến hành cập nhập dòng đó
            else: 
                x = df_warehouse.loc[lab]['LƯỢNG TỒN KHO']
                df_warehouse.at[lab, 'LƯỢNG TỒN KHO'] = x - (demand - cumulative_demand)
                z = 1
                break


    #Còn lại thì xuất theo FEFO
    if z != 1:
        for lab, row in df_warehouse[df_warehouse['ITEM'] == plan_material].sort_values('HẠN SỬ DỤNG', ascending=True).iterrows():

            
            #Nếu mà demand tích lũy <= lượng tồn kho thì tiến hành xóa dòng đó đi
            if row['LƯỢNG TỒN KHO'] + cumulative_demand <= demand:
                df_warehouse.drop(lab, inplace=True)
                cumulative_demand = cumulative_demand + row['LƯỢNG TỒN KHO']
                #Nếu không thì tiến hành cập nhập dòng đó
            else: 
                x = df_warehouse.loc[lab]['LƯỢNG TỒN KHO']
                df_warehouse.at[lab, 'LƯỢNG TỒN KHO'] = x - (demand - cumulative_demand)
                break
    start_date = start_date + datetime.timedelta(days=1)

df_MRP

C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_MRP = df_MRP.append({'Attribute':'Inventory On-hand', 'MRP_date':start_date, 'MRP_value':inventory}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  demand = df_MRP[df_MRP['Attribute'] == 'Gross requirements'][df_MRP['MRP_date'] == start_date]['MRP_value'].item()
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_MRP = df_MRP.append({'Attribute':'Net Requirements', 'MRP_date':start_date, 'MRP_value':net_requirements if net_requirements >= 0 else 0}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_14372

5
9

C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  demand = df_MRP[df_MRP['Attribute'] == 'Gross requirements'][df_MRP['MRP_date'] == start_date]['MRP_value'].item()
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_MRP = df_MRP.append({'Attribute':'Net Requirements', 'MRP_date':start_date, 'MRP_value':net_requirements if net_requirements >= 0 else 0}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_MRP = df_MRP.append({'Attribute':'Planned Receipt', 'MRP_date':start_date, 'MRP_value':net_requirements}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_

C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_MRP = df_MRP.append({'Attribute':'Inventory On-hand', 'MRP_date':start_date, 'MRP_value':inventory}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  demand = df_MRP[df_MRP['Attribute'] == 'Gross requirements'][df_MRP['MRP_date'] == start_date]['MRP_value'].item()
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_MRP = df_MRP.append({'Attribute':'Net Requirements', 'MRP_date':start_date, 'MRP_value':net_requirements if net_requirements >= 0 else 0}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_14372

9
10


C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_MRP = df_MRP.append({'Attribute':'Inventory On-hand', 'MRP_date':start_date, 'MRP_value':inventory}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  demand = df_MRP[df_MRP['Attribute'] == 'Gross requirements'][df_MRP['MRP_date'] == start_date]['MRP_value'].item()
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_MRP = df_MRP.append({'Attribute':'Gross requirements', 'MRP_date':start_date, 'MRP_value':0}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:24: FutureWarning: The frame.a

9


C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  warehouse_import_quantity  = df_MRP[df_MRP['MRP_date'] == start_date][df_MRP['Attribute'].isin(['Scheduled Receipt', 'Planned Receipt'])]['MRP_value'].sum()
C:\Users\admin\AppData\Local\Temp\ipykernel_14372\2127244020.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_warehouse = df_warehouse.append({'ITEM':plan_material, 'LƯỢNG TỒN KHO':warehouse_import_quantity, 'ĐƠN VỊ':material_unit, 'HẠN SỬ DỤNG':material_expired_date, 'NGÀY NHẬP KHO':start_date}, ignore_index=True)


,Attribute,MRP_date,MRP_value
0,Gross requirements,2023-01-01,6000
1,Gross requirements,2023-01-03,6800
2,Gross requirements,2023-01-05,8000
3,Gross requirements,2023-01-07,6800
4,Gross requirements,2023-01-10,6000
5,Scheduled Receipt,2023-01-05,1000
6,Inventory On-hand,2023-01-01,12000
7,Net Requirements,2023-01-01,0
8,Planned Receipt,2023-01-01,0
9,Schedule Releases,2022-12-25,0


In [440]:
pd.pivot_table(df_MRP, values='MRP_value', index='Attribute', columns='MRP_date')

MRP_date,2022-12-25,2022-12-26,2022-12-27,2022-12-28,2022-12-29,2022-12-30,2022-12-31,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08,2023-01-09,2023-01-10
Attribute,,,,,,,,,,,,,,,,,
Gross requirements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6000.0,0.0,6800.0,0.0,8000.0,0.0,6800.0,0.0,0.0,6000.0
Inventory On-hand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12000.0,6000.0,6000.0,2200.0,2200.0,1000.0,1000.0,0.0,0.0,0.0
Net Requirements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,800.0,0.0,5800.0,0.0,5800.0,0.0,0.0,6000.0
Planned Receipt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,3000.0,0.0,5800.0,0.0,5800.0,0.0,0.0,6000.0
Schedule Releases,0.0,0.0,3000.0,0.0,5800.0,0.0,5800.0,0.0,0.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Scheduled Receipt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN


In [441]:
df_warehouse

,ITEM,LƯỢNG TỒN KHO,ĐƠN VỊ,HẠN SỬ DỤNG,NGÀY NHẬP KHO,NGÀY ĐÃ TỒN KHO
0,KHOAI,10000,kg,2023-02-12,2022-11-04,67 days
1,SẮN,3000,kg,2023-02-27,2022-11-19,52 days
2,XƯƠNG_ĐỘNG_VẬT,2000,kg,2023-01-26,2022-12-27,14 days
3,BỘT_ĐỖ_TƯƠNG,9000,kg,2023-02-20,2022-11-12,59 days
4,MEN_VI_SINH,200,lít,2023-01-25,2022-12-26,15 days
5,THÓC,13000,kg,2023-03-03,2022-11-23,48 days
6,BAO_BÌ_ĐÓNG_GÓI,1220,cái,NaT,NaT,NaT
7,THỨC_ĂN_CHO_HEO,7000,kg,2023-01-27,2022-10-19,83 days
8,THỨC_ĂN_CHO_CÁ,8000,kg,2023-01-30,2022-10-22,80 days
